<a href="https://colab.research.google.com/github/yeshitha-kuppireddy/mancala/blob/main/mancala.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

changing board according to i

#board
#[p1marbles]
#[p1] ...... [p2]
#[p2marbles]

In [ ]:

p1mancala=0
p1marbles=[4,4,4,4,4,4]
p2mancala=0
p2marbles=[4,4,4,4,4,4]

In [ ]:
turn='p2'
i=3
change_board_acc_to_button(turn,i,p1mancala, p1marbles, p2mancala, p2marbles)

(False, 0, [4, 4, 4, 4, 4, 5], 0, [4, 4, 4, 0, 4, 4])

In [ ]:
def change_board_acc_to_button(turn, i, p1mancala, p1marbles, p2mancala, p2marbles):
    extramove = False  # Initialize extra move as False

    if turn == 'p2':
        marbles = p2marbles[i]  # Get the number of marbles from the selected pit
        p2marbles[i] = 0  # Set the selected pit to 0
        current_idx = i  # Start from the selected index

        while marbles > 0:
            current_idx += 1
            if current_idx == len(p2marbles):  # Distribute in p2's Mancala
                p2mancala += 1
                if marbles == 1:  # If last marble goes into Mancala, extra move
                    extramove = True
            elif current_idx > len(p2marbles) and current_idx <= len(p2marbles) + len(p1marbles):  # Distribute in p1's pits
                reverse_idx = len(p1marbles) - (current_idx - len(p2marbles))  # Reverse distribution in p1's pits
                p1marbles[reverse_idx] += 1
            elif current_idx > len(p2marbles) + len(p1marbles):  # Wrap around back to p2's pits
                current_idx = -1  # Reset index to wrap around

            marbles -= 1  # Decrease marble count

    elif turn == 'p1':
        marbles = p1marbles[i]  # Get the number of marbles from the selected pit
        p1marbles[i] = 0  # Set the selected pit to 0
        current_idx = i  # Start from the selected index

        while marbles > 0:
            current_idx += 1
            if current_idx == len(p1marbles):  # Distribute in p1's Mancala
                p1mancala += 1
                if marbles == 1:  # If last marble goes into Mancala, extra move
                    extramove = True
            elif current_idx > len(p1marbles) and current_idx <= len(p1marbles) + len(p2marbles):  # Distribute in p2's pits
                reverse_idx = len(p2marbles) - (current_idx - len(p1marbles))  # Reverse distribution in p2's pits
                p2marbles[reverse_idx] += 1
            elif current_idx > len(p1marbles) + len(p2marbles):  # Wrap around back to p1's pits
                current_idx = -1  # Reset index to wrap around

            marbles -= 1  # Decrease marble count

    return extramove, p1mancala, p1marbles, p2mancala, p2marbles



defining i (which move to make)

In [ ]:
def makemove(turn,p1mancala, p1marbles, p2mancala,p2marbles):
  if turn=='p2':
    print('your turn')
    i=int(input('which move do you want to make: '))
    extramove,p1mancala, p1marbles, p2mancala, p2marbles=changeboardacctobutton(turn,i,p1mancala, p1marbles, p2mancala, p2marbles)
    return extramove,p1mancala, p1marbles, p2mancala, p2marbles
  else:
    i=bestmove(p1mancala, p1marbles, p2mancala, p2marbles)
    p1mancala, p1marbles, p2mancala, p2marbles=changeboardacctobutton(turn,i,p1mancala, p1marbles, p2mancala, p2marbles)

In [ ]:
class GameState:
    def __init__(self, p1mancala, p1marbles, p2mancala, p2marbles, turn):
        self.p1mancala = p1mancala
        self.p1marbles = p1marbles[:]
        self.p2mancala = p2mancala
        self.p2marbles = p2marbles[:]
        self.turn = turn

    def is_terminal(self):
        return all(m == 0 for m in self.p1marbles) or all(m == 0 for m in self.p2marbles)

    def get_legal_moves(self):
        if self.turn == 'p1':
            return [i for i in range(6) if self.p1marbles[i] > 0]
        else:
            return [i for i in range(6) if self.p2marbles[i] > 0]

    def apply_move(self, move):
        new_state = GameState(self.p1mancala, self.p1marbles, self.p2mancala, self.p2marbles, self.turn)
        if new_state.turn == 'p1':
            extramove, new_state.p1mancala, new_state.p1marbles, new_state.p2mancala, new_state.p2marbles = changeboardacctobutton(new_state.turn,move, new_state.p1mancala, new_state.p1marbles, new_state.p2mancala, new_state.p2marbles)
            new_state.turn = 'p1' if extramove else 'p2'
        else:
            extramove, new_state.p1mancala, new_state.p1marbles, new_state.p2mancala, new_state.p2marbles = changeboardacctobutton(new_state.turn,move, new_state.p2mancala, new_state.p2marbles, new_state.p1mancala, new_state.p1marbles)
            new_state.turn = 'p2' if extramove else 'p1'
        return new_state

    def get_winner(self):
        if self.is_terminal():
            if self.p1mancala > self.p2mancala:
                return 'p1'
            elif self.p1mancala < self.p2mancala:
                return 'p2'
            else:
                return 'draw'
        return None


In [ ]:
import math
class MCTSNode:
    def __init__(self, state, parent=None):
        self.state = state
        self.parent = parent
        self.children = []
        self.visits = 0
        self.wins = 0

    def is_fully_expanded(self):
        return len(self.children) == len(self.state.get_legal_moves())

    def best_child(self, c_param=math.sqrt(2)):
        choices_weights = []
        for child in self.children:
            if child.visits == 0:
                choices_weights.append(float('inf'))  # Ensure nodes with zero visits are explored
            else:
                choices_weights.append(
                    (child.wins / child.visits) + c_param * math.sqrt((2 * math.log(self.visits) / child.visits))
                )
        return self.children[choices_weights.index(max(choices_weights))]

    def expand(self):
        legal_moves = self.state.get_legal_moves()
        for move in legal_moves:
            new_state = self.state.apply_move(move)
            child_node = MCTSNode(new_state, parent=self)
            self.children.append(child_node)

    def backpropagate(self, result):
        self.visits += 1
        if self.state.turn == 'p1' and result == 'p1':
            self.wins += 1
        elif self.state.turn == 'p2' and result == 'p2':
            self.wins += 1
        elif result == 'draw':
            self.wins += 0.5
        if self.parent:
            self.parent.backpropagate(result)


In [ ]:
import random

class MCTS:
    def __init__(self, time_limit=1):
        self.time_limit = time_limit

    def search(self, initial_state):
        root = MCTSNode(initial_state)

        import time
        end_time = time.time() + self.time_limit
        print(end_time)

        while time.time() < end_time:
            node = self.select(root)
            if not node.state.is_terminal():
                node.expand()
                node = random.choice(node.children)
            result = self.simulate(node)
            node.backpropagate(result)

        return root.best_child(c_param=0)

    def select(self, node):
        while not node.state.is_terminal() and node.is_fully_expanded():
            node = node.best_child()
        return node

    def simulate(self, node):
        current_state = node.state
        while not current_state.is_terminal():
            move = random.choice(current_state.get_legal_moves())
            current_state = current_state.apply_move(move)
        return current_state.get_winner()


In [ ]:
def bestmove(p1mancala, p1marbles, p2mancala, p2marbles):
  initial_state = GameState(p1mancala, p1marbles, p2mancala, p2marbles, turn='p1')

  mcts = MCTS(time_limit=1)  # Run MCTS for 1 second
  best_move_node = mcts.search(initial_state)

  # Extract the best move from the resulting node
  for child in best_move_node.parent.children:
      if child == best_move_node:
          best_move = initial_state.get_legal_moves()[best_move_node.parent.children.index(child)]
          break

  return best_move

p1mancala=0
p1marbles=[4,4,4,4,4,4]
p2mancala=0
p2marbles=[4,4,4,4,4,4]
bestmove(p1mancala, p1marbles, p2mancala, p2marbles)


IndexError: list index out of range

gameloop

In [ ]:
#turnitemmakesamove

turn='p2'
keys={'p1':'p2','p2':'p1'}
gameend=1

p1mancala=0
p1marbles=[4,4,4,4,4,4]
p2mancala=0
p2marbles=[4,4,4,4,4,4]
def gameend(p1mancala, p1marbles, p2mancala, p2marbles):
  if all(m == 0 for m in p1marbles) or all(m == 0 for m in p2marbles):
        return 0
  return 1

while gameend(p1mancala, p1marbles, p2mancala, p2marbles) > 0:
  extramove,p1mancala, p1marbles, p2mancala, p2marbles = makemove(turn,p1mancala, p1marbles, p2mancala, p2marbles)
  print(p1mancala)
  print(p1marbles)
  print(p2mancala)
  print(p2marbles)
  if extramove==0:
    turn=keys[turn]
  gameendr=gameend(p1mancala, p1marbles, p2mancala, p2marbles)
  if gameendr == 0:
        break


your turn
which move do you want to make: 1
0
[5, 5, 4, 4, 4, 4]
1
[5, 0, 4, 4, 4, 4]


IndexError: list index out of range